In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import random
from keras.models import load_model
import pickle
import json

Using TensorFlow backend.


In [2]:
# restore all of our data structures
data = pickle.load( open( "meta.pkl", "rb" ) )
words = data['words']
classes = data['classes']
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
model = load_model('model.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 48)                2160      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1568      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 153       
Total params: 4,409
Trainable params: 4,409
Non-trainable params: 0
_________________________________________________________________


In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [54]:
# create a data structure to hold user context
context = {}
ERROR_THRESHOLD = 0.55

def classify(sentence):
    # generate probabilities from the model
    p = np.array([bow(sentence, words)])
    results = list(model.predict(p))[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']
                    
                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i:
                        if show_details: print ('tag:', i['tag'])
                        return random.choice(i['responses'])
                    elif (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        context.pop(userID, None)
                        return random.choice(i['responses'])
            results.pop(0)
    return "Sorry, please try again"

In [55]:
classify('is your shop open today?')

[]

In [32]:
response('is your shop open today?')

'Sorry, please try again'

In [33]:
response('do you take cash?')

'Sorry, please try again'

In [35]:
response('who are you?')

"I'm HeimDall, your CyberSecurity Assistant"

In [36]:
response('Goodbye, see you later')

'Have a nice day'

In [37]:
context

{'123': 'gen_cert'}

In [58]:
response('we want to rent a moped')

'Sorry, please try again'

In [57]:
# show context
context

{}

In [40]:
response('today')

'Sorry, please try again'

In [41]:
classify('today')

[]

In [42]:
# clear context
response("Hi there!", show_details=True)

context: 
tag: greeting


'Hello, thanks for visiting'

In [20]:
response('today')
classify('today')

[]

In [46]:
response("thanks, your great")

'Happy to help!'

In [59]:
response("who created you")

'I was created by Kanak Dahake Jr.'

In [60]:
convo = 'hello there bye'
print(classify(convo))
print(response(convo))

[('greeting', 0.9629196)]
Hi there, how can I help?


In [63]:
convo = 'vm'
print(context)
print(classify(convo))
print(response(convo, show_details=True))

{'123': ''}
[('gen_cert_type', 0.88656294)]
Sorry, please try again


In [74]:
convo = 'generate certificates'
print(classify(convo))
print(response(convo, show_details=True))

[('gen_cert', 0.9390314)]
context: gen_cert
tag: gen_cert
Do you want to generate certificate for Resource group or VM?


In [78]:
convo = 'vm'
print(context)
print(classify(convo))
print(response(convo, show_details=True))

{}
[('gen_cert_type', 0.88656294)]
Sorry, please try again
